In [58]:
import pandas as pd
import requests
import numpy as np 

In [ ]:
df_booking = pd.read_csv("../data/raw/bookings.csv")
df_grounds = pd.read_excel("../data/raw/grounds.xlsx")

In [53]:
url = "https://jsonplaceholder.typicode.com/users"
users = requests.get(url).json()
df_users = pd.json_normalize(users)

In [54]:
# Preview
# print(df_booking.head())
# print(df_grounds.head())
print(df_users.head())

   id              name   username                      email  \
0   1     Leanne Graham       Bret          Sincere@april.biz   
1   2      Ervin Howell  Antonette          Shanna@melissa.tv   
2   3  Clementine Bauch   Samantha         Nathan@yesenia.net   
3   4  Patricia Lebsack   Karianne  Julianne.OConner@kory.org   
4   5  Chelsey Dietrich     Kamren   Lucio_Hettinger@annie.ca   

                   phone        website     address.street address.suite  \
0  1-770-736-8031 x56442  hildegard.org        Kulas Light      Apt. 556   
1    010-692-6593 x09125  anastasia.net      Victor Plains     Suite 879   
2         1-463-123-4447    ramiro.info  Douglas Extension     Suite 847   
3      493-170-9623 x156       kale.biz        Hoeger Mall      Apt. 692   
4          (254)954-1289   demarco.info       Skiles Walks     Suite 351   

    address.city address.zipcode address.geo.lat address.geo.lng  \
0    Gwenborough      92998-3874        -37.3159         81.1496   
1    Wisokyburgh

In [56]:
print(df_booking.head())
print(df_grounds.head())

   booking_id  user_id  ground_id booking_date  hours  total_price  rating  \
0        1001      201        301   2025-01-05    2.0          NaN     4.0   
1        1002      202        302   2025-01-05    1.0       1200.0     NaN   
2        1003      203        301   2025-01-06    3.0       3000.0     3.0   
3        1004      204        303   2025-01-06    2.0       2200.0     4.0   
4        1005      201        302   2025-01-07    1.0       1200.0     2.0   

  payment_method  
0          esewa  
1         khalti  
2            NaN  
3           card  
4          esewa  
   ground_id     ground_name       city location_type  price_per_hour indoor  \
0        301      Goal Arena  Kathmandu        indoor          1000.0    yes   
1        302     KickOff Hub   Lalitpur       outdoor          1200.0     no   
2        303      Futsal Pro  Bhaktapur        indoor             NaN    yes   
3        304  Champions Turf  Kathmandu       outdoor          1300.0     no   
4        305    U

In [59]:
num_cols = df_booking.select_dtypes(include=np.number).columns
print(num_cols)

Index(['booking_id', 'user_id', 'ground_id', 'hours', 'total_price', 'rating'], dtype='object')


In [60]:
for col in num_cols:
    df_booking[col] = df_booking[col].fillna(df_booking[col].median())

In [61]:
df_booking

,booking_id,user_id,ground_id,booking_date,hours,total_price,rating,payment_method
0,1001,201,301,2025-01-05,2.0,2000.0,4.0,esewa
1,1002,202,302,2025-01-05,1.0,1200.0,4.0,khalti
2,1003,203,301,2025-01-06,3.0,3000.0,3.0,NaN
3,1004,204,303,2025-01-06,2.0,2200.0,4.0,card
4,1005,201,302,2025-01-07,1.0,1200.0,2.0,esewa
5,1006,205,304,2025-01-07,2.0,2600.0,5.0,khalti
6,1007,206,301,2025-01-08,2.0,2000.0,4.0,cash
7,1008,202,303,2025-01-08,1.0,1100.0,3.0,card


In [62]:
cat_cols = df_booking.select_dtypes(include='object').columns

In [63]:
# Fill NaN in categorical columns with mode (most frequent value)
for col in cat_cols:
    df_booking[col] = df_booking[col].fillna(df_booking[col].mode()[0])

In [64]:
df_booking

,booking_id,user_id,ground_id,booking_date,hours,total_price,rating,payment_method
0,1001,201,301,2025-01-05,2.0,2000.0,4.0,esewa
1,1002,202,302,2025-01-05,1.0,1200.0,4.0,khalti
2,1003,203,301,2025-01-06,3.0,3000.0,3.0,card
3,1004,204,303,2025-01-06,2.0,2200.0,4.0,card
4,1005,201,302,2025-01-07,1.0,1200.0,2.0,esewa
5,1006,205,304,2025-01-07,2.0,2600.0,5.0,khalti
6,1007,206,301,2025-01-08,2.0,2000.0,4.0,cash
7,1008,202,303,2025-01-08,1.0,1100.0,3.0,card


In [65]:
df_booking['booking_date'] = pd.to_datetime(df_booking['booking_date'], errors='coerce')

In [66]:
df_booking

,booking_id,user_id,ground_id,booking_date,hours,total_price,rating,payment_method
0,1001,201,301,2025-01-05,2.0,2000.0,4.0,esewa
1,1002,202,302,2025-01-05,1.0,1200.0,4.0,khalti
2,1003,203,301,2025-01-06,3.0,3000.0,3.0,card
3,1004,204,303,2025-01-06,2.0,2200.0,4.0,card
4,1005,201,302,2025-01-07,1.0,1200.0,2.0,esewa
5,1006,205,304,2025-01-07,2.0,2600.0,5.0,khalti
6,1007,206,301,2025-01-08,2.0,2000.0,4.0,cash
7,1008,202,303,2025-01-08,1.0,1100.0,3.0,card
